# **Desarrollo de Interfaz Gráfica**

### Con el fin de facilitar el uso de la aplicación, se desarrolló una interfaz gráfica que permite al usuario interactuar con la misma de manera más amigable. Cabe destacar que el código implementado es el mismo que se encuentra en el archivo [Local_Endpoints](5-Local_Endpoints.ipynb) a efectos que los resultados deriven de la consulta a los datasets locales eliminando costes a los que se pudiesen incurrir si estuviese conectada al datalake de AWS. Posterior a esto, se puede empaquetar la aplicación en un archivo ejecutable para que el usuario pueda utilizarla sin necesidad de tener instalado Python en su computadora mediante librerias como [PyInstaller](https://www.pyinstaller.org/), [Kivy](https://kivy.org/#home), [PyQt](https://www.riverbankcomputing.com/software/pyqt/intro) u otras. En este caso se optó por PyInstaller y se puede acceder a una copia del ejecutable en el siguiente [enlace](https://drive.google.com/file/d/1xS1JglCGvLJ82eEp4-zb3IeixiZlT8iC/view?usp=sharing). `Observación:` El .exe se detecta como amenaza al no contar con firma digital por lo que se debe agregar una excepción en el antivirus en caso de querer ejecutarlo.

### Importamos las librerías necesarias.

In [3]:
import tkinter as tk
from tkinter import ttk, Entry, Label, Button, messagebox
import pandas as pd
from datetime import timedelta
import joblib
import os

### **`Funciones, modelo y entorno gráfico`**

In [4]:
class InputData:
    def __init__(self, pBoroughID, dayofweek, hour):
        self.pBoroughID = pBoroughID
        self.dayofweek = dayofweek
        self.hour = hour

def predict(data):
    # Leemos el modelo entrenado
    xgb_model_path = 'xgb_model.pkl'

    with open(xgb_model_path, 'rb') as model_file:
        xgb_model = joblib.load(model_file)

    # Convertimos los datos de entrada en un DataFrame
    input_data = pd.DataFrame([vars(data)])

    # Realizamos la predicción
    prediction = xgb_model.predict(input_data)

    # Retornamos el resultado formateado como texto
    result_text = f"Probability of getting a passenger: {prediction[0]:.2f}"
    return result_text

def top_3_vehicles(max_price_usd):
    # Leemos el DataFrame desde el archivo parquet
    vehicles_info_path = 'Datasets/3-Normalizados/vehicles_info.parquet'  # Ruta relativa

    vehicles_info = pd.read_parquet(vehicles_info_path)

    # Filtramos los vehículos por precio
    filtered_vehicles = vehicles_info[vehicles_info['Price (USD)'] <= max_price_usd]

    # Ordenamos los vehículos por emisiones de CO2, emisiones de sonido y rango de autonomía
    sorted_vehicles = filtered_vehicles.sort_values(by=['CO2 Emission (g/mi)', 'Sound Emission (dB)', 'Range (mi)'], ascending=[True, True, False])

    # Tomamos los 3 primeros vehículos
    top_3_vehicles = sorted_vehicles.head(3)

    # Formateamos el resultado como texto
    result_text = "\n".join([f"{row.Manufacturer} {row.Model} - Price: ${row._9:.2f}, Fuel: {row.Fuel}, CO2: {row._7}, dB: {row._8}, Range with a full tank: {row._10}" for row in top_3_vehicles.itertuples()])
    return result_text

def get_taxi_stats(pickup_borough):
    # Leemos el DataFrame desde el archivo parquet
    taxis_info_path = 'Datasets/3-Normalizados/taxis_2023.parquet'  # Ruta relativa

    taxis = pd.read_parquet(taxis_info_path)

    # Filtramos los taxis por borough
    filtered_taxis = taxis[taxis['pickup_borough'] == pickup_borough]

    # Calculamos el número total de viajes
    total_trips = len(filtered_taxis)

    # Calculamos la duración media de los viajes
    duration_timedelta = (filtered_taxis['dropoff_datetime'] - filtered_taxis['pickup_datetime'])
    average_trip_duration = round(duration_timedelta.mean() / timedelta(hours=1), 2)

    # Calculamos el número medio de viajes por día
    daily_trips = round(filtered_taxis.groupby(filtered_taxis['pickup_datetime'].dt.date).size().mean(), 2)

    # Calculamos el número medio de viajes por mes
    monthly_trips = round(filtered_taxis.groupby(filtered_taxis['pickup_datetime'].dt.to_period("M")).size().mean(), 2)

    average_trip_distance = round(filtered_taxis['trip_distance'].mean(), 2)
    average_total_amount = round(filtered_taxis['total_amount'].mean(), 2)

    # Formateamos el resultado como texto
    result_text = (
        f"Total Trips: {total_trips}\n"
        f"Average Trip Duration (Hrs): {average_trip_duration}\n"
        f"Mean Trips per Day: {daily_trips}\n"
        f"Mean Trips per Month: {monthly_trips}\n"
        f"Average Trip Distance (miles): {average_trip_distance}\n"
        f"Total Earned on Average (USD): {average_total_amount}"
    )
    return result_text


# Leemos el modelo entrenado
xgb_model_path = 'xgb_model.pkl'
with open(xgb_model_path, 'rb') as model_file:
    xgb_model = joblib.load(model_file)



# Creamos la ventana principal
root = tk.Tk()
root.title("daTest")
root.resizable(False, False)  # Desactivamos la posibilidad de redimensionar

# Cargamos el ícono de la ventana principal
icon_path_root = 'Datasets/4-Extras/datagnius.ico' 

# Verificamos si el archivo del ícono existe antes de intentar cargarlo
if os.path.exists(icon_path_root):
    root.iconbitmap(icon_path_root)
else:
    messagebox.showwarning("Warning", "Icon file 'datagnius.ico' not found.")

# Funciones para mostrar las ventanas secundarias
def show_result_window(title, result_text):
    result_window = tk.Toplevel(root)
    result_window.title(title)

    # Configuramos el ícono usando la ruta completa al archivo del ícono en la ventana secundaria
    icon_path_result = 'Datasets/4-Extras/datagnius.ico'

    if os.path.exists(icon_path_result):
        result_window.iconbitmap(icon_path_result)

    result_label = Label(result_window, text=result_text, padx=10, pady=10)
    result_label.pack()


# Función para mostrar la ventana "Created By"
def show_created_by():
    created_by_window = tk.Toplevel(root)
    created_by_window.title("Created By")

    created_by_label = Label(created_by_window, text="Created by datagnius", font=("Helvetica", 12))
    created_by_label.pack(padx=10, pady=5)

def on_predict_button():
    try:
        pBoroughID = int(entry_borough.get())
        dayofweek = int(entry_day.get())
        hour = int(entry_hour.get())

        input_data = InputData(pBoroughID=pBoroughID, dayofweek=dayofweek, hour=hour)
        result = predict(input_data)

        show_result_window("Prediction Result", result)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

def on_top_3_vehicles_button():
    try:
        max_price_usd = float(entry_max_price.get())
        result = top_3_vehicles(max_price_usd)

        show_result_window("Top 3 Vehicles", result)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

def on_taxi_stats_button():
    try:
        pickup_borough = entry_pickup_borough.get()
        result = get_taxi_stats(pickup_borough)

        show_result_window("Taxi Statistics", result)
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Estilo de la ventana principal
style = ttk.Style()

# Estilo del título
style.configure("Title.TFrame", background="#FFA500")

title_style = {"font": ("Helvetica", 14, "bold")}

# Primer sector: Passenger Prediction
frame_passenger = ttk.Frame(root, padding=(10, 5))
frame_passenger.grid(row=0, column=0, columnspan=2, sticky="ew")
frame_passenger["style"] = "Title.TFrame"

Label(frame_passenger, text="Passenger Prediction", **title_style).grid(row=0, column=0, columnspan=2)

Label(frame_passenger, text="Borough:").grid(row=1, column=0, padx=10, pady=5)
entry_borough = Entry(frame_passenger)
entry_borough.grid(row=1, column=1, padx=10, pady=5)

Label(frame_passenger, text="Day:").grid(row=2, column=0, padx=10, pady=5)
entry_day = Entry(frame_passenger)
entry_day.grid(row=2, column=1, padx=10, pady=5)

Label(frame_passenger, text="Hour:").grid(row=3, column=0, padx=10, pady=5)
entry_hour = Entry(frame_passenger)
entry_hour.grid(row=3, column=1, padx=10, pady=5)

Button(frame_passenger, text="Predict", command=on_predict_button).grid(row=4, column=0, columnspan=2, pady=10)

# Divisor
ttk.Separator(root, orient="horizontal").grid(row=1, column=0, columnspan=2, sticky="ew")

# Segundo sector: Vehicle Recommendation
frame_vehicles = ttk.Frame(root, padding=(10, 5))
frame_vehicles.grid(row=2, column=0, columnspan=2, sticky="ew")
frame_vehicles["style"] = "Title.TFrame"

Label(frame_vehicles, text="Vehicle Recommendation", **title_style).grid(row=0, column=0, columnspan=2)

Label(frame_vehicles, text="Max Price (USD):").grid(row=1, column=0, padx=10, pady=5)
entry_max_price = Entry(frame_vehicles)
entry_max_price.grid(row=1, column=1, padx=10, pady=5)

Button(frame_vehicles, text="Top 3 Vehicles", command=on_top_3_vehicles_button).grid(row=2, column=0, columnspan=2, pady=10)

# Divisor
ttk.Separator(root, orient="horizontal").grid(row=4, column=0, columnspan=2, sticky="ew")

# Tercer sector: Travel Statistics
frame_statistics = ttk.Frame(root, padding=(10, 5))
frame_statistics.grid(row=5, column=0, columnspan=2, sticky="ew")
frame_statistics["style"] = "Title.TFrame"

Label(frame_statistics, text="Travel Statistics", **title_style).grid(row=0, column=0, columnspan=2)

Label(frame_statistics, text="Pickup Borough:").grid(row=1, column=0, padx=10, pady=5)
entry_pickup_borough = Entry(frame_statistics)
entry_pickup_borough.grid(row=1, column=1, padx=10, pady=5)

Button(frame_statistics, text="Taxi Stats", command=on_taxi_stats_button).grid(row=2, column=0, columnspan=2, pady=10)

# Divisor
ttk.Separator(root, orient="horizontal").grid(row=7, column=0, columnspan=2, sticky="ew")

# Mensaje de 'Created By'
Label(root, text="Created by datagnius", font=("Helvetica", 10), fg="#888888").grid(row=8, column=0, columnspan=2, pady=5)

# Corremos la ventana principal
root.mainloop()